In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import bqplot
import ipywidgets as widgets

from IPython.display import display

import time
import random

from checkers_load import makeBatches, load_games, dataGenerator, getData
from checkers_display import BoardDisplay, SimpleBoardDisplay, MoveHeatmapDisplay

from checkers_helpers import *

In [2]:
pos = getDefaultPosition()

figs = []
if True:
    b = BoardDisplay(canInteract=True)
    b.update(pos)
    
    figs.append(b.fig)

if False:
    a = SimpleBoardDisplay()
    a.update(pos)
    figs.append(a.fig)
if True:
    m = MoveHeatmapDisplay()
    figs.append(m.fig)
    
rawMoves = np.ones((8,4,4)) #np.random.random((8,4,4))

display(widgets.HBox(figs))

5.0
0.1


In [3]:
def displayInputOutput(x_i, y_i):
    b.update(x_i)
    m.update(y_i)
i = 0

In [6]:
displayInputOutput(X[i], y[i])
i += 1

In [5]:
games = load_games("22k_checkers.txt")


X, y = getData(games)
n = X.shape[0]

X2 = np.zeros((n,8,8,1))
#print(m.shape, m[::2, ::2].shape, m[1::2, 1::2].shape, x0[::2, :].shape, x0[1::2, :].shape)
X2[:, ::2, 1::2] = X[:, ::2, :]
X2[:, 1::2, ::2] = X[:, 1::2, :]

Size of input is: 1521.05984 MB.


In [ ]:
for i in range(n):

    if i % 100000 == 0:
        print("training ix:", i)

    displayInputOutput(X[i], y[i])
    time.sleep(0.1)

In [ ]:
# Test: Show most popular moves.
xsum = np.zeros((8,4))
ysum = np.zeros((8,4,4))
for i, (x_i, y_i) in enumerate(dataGenerator(games)):  # TODO: Show better
    xsum += x_i
    ysum += y_i

ynorm = ysum / np.amax(ysum)

displayInputOutput(xsum, ynorm)


In [ ]:
# DO ML!!

import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

In [9]:
from checkers_models import getModel, metrics

scale = 32
#model = getModel(scale)
# Shit to load metrics *sigh*
dependencies = {
    'top3Accuracy': metrics[2],
    'top5Accuracy': metrics[3]
}

model = keras.models.load_model('trained/stupidmodel-softmax-layers5.h5', custom_objects=dependencies)


NameError: name 'keras' is not defined

In [ ]:
model.summary()

In [ ]:
# Create UI with 3 boards: Game state, target output, network output.
figs = []

board_d = BoardDisplay(canInteract=False)
figs.append(board_d.fig)

move_d = MoveHeatmapDisplay()
figs.append(move_d.fig)

pred_d = MoveHeatmapDisplay()
figs.append(pred_d.fig)
    
display(widgets.HBox(figs))

In [ ]:
ix = random.randrange(0,10000)

#for ix in range(30):
x_i, y_i = X2[ix], y[ix]

x = np.expand_dims(x_i, axis=0)
y_pred = model.predict([x])[0]

#ymax = np.amax(y_pred)
#y_pred = y_pred / ymax


board_d.update(X[ix])
move_d.update(y_i)
pred_d.update(y_pred)